In [ ]:
# Ingredients to be used in 10 recipes

# ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]
# ["bay scallops", "butter", "onions", "parsley", "lemons", "bread crumbs"]
# ["rice", "chicken", "cream of mushroom soup", "cream of chicken soup", "onion soup", "water"]
# ["coriander", "allspice", "cumin", "ground ginger", "turmeric", "cinnamon", "cayenne", "pepper", "salt", "lemon juice", "olive oil", "garlic", "chicken", "pita", "red onion", "parsley", "shredded red cabbage", "tahini"]
# ["sour cream", "heavy cream", "baking powder", "kosher salt", "sugar", "ground black pepper", "flour", "butter", "salt", "cutter"]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [1]:
import pandas as pd
import os

from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import mysql.connector

In [2]:

current_path = os.getcwd()
print(current_path)

os.chdir('../dataset/')
path_base_dataset = os.getcwd()
print(path_base_dataset)

os.chdir('../exploratory_data_analysis/')
path_base_notebook = os.getcwd()
print(path_base_notebook)

NUM_DATA_SAMPLES = 50000
file_name = '/sampled_dataset_' + str(NUM_DATA_SAMPLES) + '.csv'
sampled_data_file_path = path_base_dataset + file_name


c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\exploratory_data_analysis
c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\dataset
c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\exploratory_data_analysis


In [3]:
df = pd.read_csv(sampled_data_file_path)
df.head()

,Unnamed: 0,ID,name,ingredients,procedure,link,source,NER
0,1254789,1254789,"Bacon, Lettuce, Tomato And Fried Egg Sandwiches","[""4 slices bacon, halved"", ""4 None eggs"", ""4 N...","[""Lightly coat a nonstick frying pan and place...",recipes-plus.com/api/v2.0/recipes/37658,Gathered,"[""bacon"", ""eggs"", ""buns"", ""tomatoes"", ""mayonna..."
1,2162678,2162678,Sauteed Bay Scallops Recipe,"[""2 pound bay scallops"", ""1/4 pound butter, me...","[""Saute/fry onions and parsley in butter."", ""R...",cookeatshare.com/recipes/sauteed-bay-scallops-...,Recipes1M,"[""bay scallops"", ""butter"", ""onions"", ""parsley""..."
2,832049,832049,No Fuss No Mess Chicken And Rice,"[""1 c. uncooked rice"", ""1 chicken, cut up"", ""1...","[""Place rice in oblong pan."", ""Place chicken o...",www.cookbooks.com/Recipe-Details.aspx?id=334186,Gathered,"[""rice"", ""chicken"", ""cream of mushroom soup"", ..."
3,1396186,1396186,Baked Chicken Shawarma,"[""2 teaspoons coriander"", ""1 teaspoon allspice...","[""In a large bowl combine coriander, allspice,...",food52.com/recipes/41562-baked-chicken-shawarma,Gathered,"[""coriander"", ""allspice"", ""cumin"", ""ground gin..."
4,1322877,1322877,Salt-And-Pepper Biscuits,"[""3/4 cup sour cream"", ""4 tablespoons heavy cr...","[""Preheat oven to 425\u00b0F. Whisk sour cream...",www.epicurious.com/recipes/food/views/salt-and...,Gathered,"[""sour cream"", ""heavy cream"", ""baking powder"",..."


In [4]:
# # Database connection details
server = 'rag_application_server'
prg_database_name = 'personalized_recipe_generator'
user = 'root'
password = 'root'
host="localhost"
port="3306"

In [5]:
# Connect to MySQL (without specifying a database)
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

cursor = connection.cursor()

# Create a new database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {prg_database_name};")
print(f"Database {prg_database_name} created successfully.")

# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()

# Create a connection engine using SQLAlchemy
db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{prg_database_name}"
engine = create_engine(db_uri)

# Write the DataFrame to the MySQL database as a table
table_name = 'recipe_knowledge_base'  # Name the table you want to create in the database
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print(f"Table {table_name} populated successfully.")

# # Now connect to the newly created database using LangChain's SQLDatabase library
# db_url = f"mysql+mysqlconnector://{user}:{password}@{host}/{new_database_name}"
# db = SQLDatabase.from_uri(db_url)

# # Example: Check tables in the new database (should be empty initially)
# print(db.get_table_names())  # Outputs an empty list as the new database is empty


Database personalized_recipe_generator created successfully.
Table recipe_knowledge_base populated successfully.


In [6]:
def init_database(user: str, password: str, host: str, port: str, database: str) -> SQLDatabase:
  db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
  return SQLDatabase.from_uri(db_uri)

# # Now, you can interact with this table using LangChain's SQLDatabase
db = init_database(
  user="root",
  password="root",
  host="localhost",
  port="3306",
  database=prg_database_name
  )

In [9]:
# Verify by printing the list of tables in the database
print(db.dialect)
print(db.get_table_names())  # This should list the newly created table


mysql
['recipe_knowledge_base']


In [10]:
db.run("SELECT * FROM recipe_knowledge_base LIMIT 10;")

'[(1254789, 1254789, \'Bacon, Lettuce, Tomato And Fried Egg Sandwiches\', \'["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves", "2 None tomatoes, sliced", "None None mayonnaise or barbecue sauce"]\', \'["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or until browned. Transfer to a plate and keep warm.", "Reduce heat to medium-low. Fry eggs until cooked to your liking.", "Top bottom burger buns with lettuce, bacon, tomato and egg. Add sauce of your...\', \'recipes-plus.com/api/v2.0/recipes/37658\', \'Gathered\', \'["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]\'), (2162678, 2162678, \'Sauteed Bay Scallops Recipe\', \'["2 pound bay scallops", "1/4 pound butter, melted", "1 c. finely minced onions", "1/2 bunch parsley, finely minced", "Juice of 2 lemons", "1 c. bread crumbs"]\', \'["Saute/fry onions and parsley in butter.", "Remove from heat.", "Pat scallops in pan till all sides are full c

In [11]:
db.get_table_info()

'\nCREATE TABLE recipe_knowledge_base (\n\t`Unnamed: 0` BIGINT, \n\t`ID` BIGINT, \n\tname TEXT, \n\tingredients TEXT, \n\t`procedure` TEXT, \n\tlink TEXT, \n\tsource TEXT, \n\t`NER` TEXT\n)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from recipe_knowledge_base table:\nUnnamed: 0\tID\tname\tingredients\tprocedure\tlink\tsource\tNER\n1254789\t1254789\tBacon, Lettuce, Tomato And Fried Egg Sandwiches\t["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves",\t["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or unt\trecipes-plus.com/api/v2.0/recipes/37658\tGathered\t["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]\n2162678\t2162678\tSauteed Bay Scallops Recipe\t["2 pound bay scallops", "1/4 pound butter, melted", "1 c. finely minced onions", "1/2 bunch parsley\t["Saute/fry onions and parsley in butter.", "Remove from heat.", "Pat scallops in pan till all sides\tcookea